## Introduction

In R, `data.table` is the go-to package for manipulating any large tabular datasets. It is widely used for fast aggregation of large datasets, low latency add/update/remove of columns, quicker ordered joins, and a fast file reader.

[Python datatable](https://github.com/h2oai/datatable) is closely related to R's `data.table` and attempts to mimic its core algorithms and API.  It is close in spirit to pandas or SFrame; however we put specific emphasis of the package is on speed and big data support. 

In this notebook, let us explore the functions present in the `datatable` module and also see how it fares in speed with other modules. 

### References : 
1. https://cran.r-project.org/web/packages/data.table/index.html
2. https://www.dezyre.com/data-science-in-r-programming-tutorial/r-data-table-tutorial
3. https://datatable.readthedocs.io/en/latest/

Python datatable is not natively present in the Kaggle environment. So let us install the module first as per the instructions in the [github page](https://github.com/h2oai/datatable).

In [1]:
!pip install https://s3.amazonaws.com/h2o-release/datatable/stable/datatable-0.8.0/datatable-0.8.0-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 10.2MB 826kB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.7MB/s eta 0:00:011


Now let us import the necessary modules.

In [2]:
import time
import numpy as np
import pandas as pd
import datatable as dt
print(dt.__version__)

0.8.0


## Reading Data

Let us first start with reading the input data from csv file. `loan.csv` file has about 2.26 million rows and 145 columns.

`fread` is the fucntion to read the files in datatable module. From the documentation,fread provides fast and convenient parsing of text (csv) files.
* Automatically detects separators, headers, column types, quoting rules, etc.
* Reads from file, URL, shell, raw text, archives, glob
* Provides multi-threaded file reading for maximum speed
* Includes a progress indicator when reading large files
* Reads both RFC4180-compliant and non-compliant files




In [3]:
## Data Table Reading
start = time.time()
dt_df = dt.fread("../input/loan.csv")
end = time.time()
print(end - start)

11.487196207046509


This took about ~10 seconds to read the data and create a `Frame` object. The fundamental unit of analysis in datatable is a data `Frame`. It is the same notion as a pandas DataFrame or SQL table: data arranged in a two-dimensional array with rows and columns.

Now let us read the same file in pandas as see the time difference.

In [4]:
import time

start = time.time()
pd_df= pd.read_csv("../input/loan.csv")
end = time.time()
print(end - start)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


51.50403642654419


Pandas takes more than ~50 seconds to read the same file and create a dataframe from it. I think this will be a good value add when we deal with large datasets. 

We also have options in datatable to convert the frame to pandas dataframe object. Let us do that. 

In [5]:
start = time.time()
dt_df.to_pandas()
end = time.time()
print(end - start)

21.696823358535767


Looks like this takes ~20 seconds. So reading a file as datable frame and converting that to pandas dataframe is faster than reading through pandas directly ;)

## Frame Properties

Similar to pandas, we can get the 

* top few rows by `head` command, 
* size of the frame by `shape` command and 
* column names by `names` command

In [6]:
dt_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,…,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,NA,NA,2500,2500,2500,36 months,13.56,84.92,C,C1,…,,,NA,NA,NA
1,NA,NA,30000,30000,30000,60 months,18.94,777.23,D,D2,…,,,NA,NA,NA
2,NA,NA,5000,5000,5000,36 months,17.97,180.69,D,D1,…,,,NA,NA,NA
3,NA,NA,4000,4000,4000,36 months,18.94,146.51,D,D2,…,,,NA,NA,NA
4,NA,NA,30000,30000,30000,60 months,16.14,731.78,C,C4,…,,,NA,NA,NA
5,NA,NA,5550,5550,5550,36 months,15.02,192.45,C,C3,…,,,NA,NA,NA
6,NA,NA,2000,2000,2000,36 months,17.97,72.28,D,D1,…,,,NA,NA,NA
7,NA,NA,6000,6000,6000,36 months,13.56,203.79,C,C1,…,,,NA,NA,NA
8,NA,NA,5000,5000,5000,36 months,17.97,180.69,D,D1,…,,,NA,NA,NA


In [7]:
# number of rows and columns
dt_df.shape

(2260668, 145)

In [8]:
# To get the column names
dt_df.names[:10]

('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade')

## Summary Statistics

We can also get the summary statistics like sum, mean, max, min, sd etc.

Let us get the mean of the columns using both datatable and pandas

In [9]:
dt_df.mean()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,…,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,NA,NA,15046.9,15041.7,15023.4,NA,13.0929,445.808,NA,NA,…,NA,NA,5030.61,47.7756,13.1486


When I ran the below one, I got memory error and so commenting it out.

In [ ]:
## pd_df.mean()

## Sorting the frame

Now let us sort the dataframe by loan_amnt column.

In [10]:
start = time.time()
dt_df.sort("loan_amnt")
end = time.time()
print(end - start)

0.0813906192779541


In [11]:
start = time.time()
pd_df.sort_values(by="loan_amnt")
end = time.time()
print(end - start)

11.98312520980835


Wow! This is super fast. 

## GroupBy Operations

Let us get the mean loan amount for each grades. 

In datatable, almost all operations with a Frame can be expressed as

`DT[ i, j, ...]`

where 

`i` is the row selector

`j` is the column selector and 

`...` represents additional modifiers might be added

In [12]:
start = time.time()
for i in range(100):
    dt_df[:, dt.sum(dt.f.loan_amnt), dt.by(dt.f.grade)]
end = time.time()
print(end - start)

4.2894606590271


In this case `i` is all the rows, `j` is loan_amnt and `...` is grouping by grade

We have also used `dt.f` in the code - what does it mean? It is a "frame proxy", as it becomes a simple way to refer to the Frame that we currently operate on. In this example, `dt.f` represents the frame `dt_df` 

In [13]:
start = time.time()
for i in range(100):
    pd_df.groupby("grade")["loan_amnt"].sum()
end = time.time()
print(end - start)

14.284939050674438


## Filtering rows

Filtering also follows the same syntax as that of groupby

Let us filter those rows which has loan_amnt greater than the mean loan_amnt.

In [14]:
start = time.time()
for i in range(100):
    dt_df[dt.f.loan_amnt>dt.mean(dt.f.loan_amnt), "loan_amnt"]
end = time.time()
print(end - start)

2.826692819595337


In [15]:
start = time.time()
for i in range(100):
    pd_df["loan_amnt"][pd_df["loan_amnt"] > pd_df["loan_amnt"].mean()]
end = time.time()
print(end - start)

4.060319662094116


## Model Building

We can also build models using the dataframe as such. 

Let us try to build a linear regression model to predict the "interest rate" from "loan amount" and "installment". We can also build other models but let us build this one is for simplicity.

Thanks Mike for the question on the comments section. 

In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(dt_df[:,["loan_amnt", "installment"]], dt_df[:,"int_rate"])
model.coef_

array([[-9.50494440e-05,  5.33344794e-03]])

## Saving the Frame

Now let us look at the ways to save the frame so as to load back when needed. `datatable` has to_csv function similar to pandas.

In [17]:
gdf = dt_df[:, dt.sum(dt.f.loan_amnt), dt.by(dt.f.grade)]
gdf.to_csv("temp.csv")

Overall, the `datatable` module seems to be much faster compared to the pandas dataframe operations. This will be very handy when we deal with bigger datasets.

One drawback is that `datatable` does not have all the functionalities that are there in pandas.